# Imports

In [225]:
import sys
sys.path.append("../")

import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

from functions import (
    import_data,
    reduce_mem_usage,
    get_df_uniques, 
    manage_major_values, 
    get_corr_pairs_thresh,
    compute_isna_percent    
)


# Getting data

In [226]:
# # Loads the California housing Dataset
# housing = fetch_openml(name="house_prices", as_frame=True)

# # Concatenate datas into a single dataframe
# data = pd.DataFrame(
#     data=np.c_[housing['data'], housing['target']],
#     columns=housing['feature_names'] + ['target']
# )
# data = data.set_index("Id")

# # Save raw datas
# data.to_csv("data/01/house_prices.csv")

# del housing, data

In [227]:
df = pd.read_csv("http://jse.amstat.org/v19n3/decock/AmesHousing.txt", sep="\t")
df.to_csv("./data/01/ames_full.csv")
print(df.shape)
df.info(verbose=False)

(2930, 82)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Columns: 82 entries, Order to SalePrice
dtypes: float64(11), int64(28), object(43)
memory usage: 1.8+ MB


# Loading dataframe

In [228]:
file = "./data/01/ames_full.csv"
# Helper for reducing memory usage of dataframe
df = import_data(file)

Memory usage of dataframe is 1.855 MB
Memory usage after optimization is: 0.36 MB
Decreased by 80.4%


In [229]:
df.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2930 entries, 0 to 2929
Columns: 82 entries, Order to SalePrice
dtypes: category(43), float16(11), int16(16), int32(3), int8(9)
memory usage: 371.9 KB


# Data cleaning

Firstly, we'll remove data not corresponding residential units and abnormal sales

In [230]:
# Delete all df with "MS Zoning" = commercial, agriculture and industrial as these are not residential units
df = df[(df["MS Zoning"] != 'C (all)') & (
    df["MS Zoning"] != 'I (all)') & (df["MS Zoning"] != 'A (agr)')]

# Delete abnormal sales
df = df[(df["Sale Condition"] != 'Abnorml')]
df = df[(df["Sale Condition"] != 'Family')]

### Most missing values by features

In [231]:
nans = compute_isna_percent(df)
nans[:10]

Pool QC          99.59
Misc Feature     96.38
Alley            93.68
Fence            80.90
Fireplace Qu     47.01
Lot Frontage     17.04
Garage Yr Blt     4.71
Garage Cond       4.71
Garage Qual       4.71
Garage Finish     4.71
dtype: float64

the features 'Pool Qc', 'MiscFeature', 'Alley', 'Fence' are missing too much values to be revelant


In [232]:
df = df.drop(["Pool QC", "Misc Feature", "Alley", "Fence"], axis=1)

## Feature engineering
### Correlated pairs

In [233]:
get_corr_pairs_thresh(df, 20, .5)

0.97571 ('Yr Sold', 'Order')
0.88601 ('Garage Area', 'Garage Cars')
0.83601 ('Garage Yr Blt', 'Year Built')
0.81123 ('TotRms AbvGrd', 'Gr Liv Area')
0.80412 ('Total Bsmt SF', '1st Flr SF')
0.80032 ('SalePrice', 'Overall Qual')
0.71233 ('Gr Liv Area', 'SalePrice')
0.66702 ('TotRms AbvGrd', 'Bedroom AbvGr')
0.65483 ('Year Remod/Add', 'Garage Yr Blt')
0.65230 ('SalePrice', 'Garage Cars')
0.65039 ('Gr Liv Area', '2nd Flr SF')
0.64931 ('SalePrice', 'Garage Area')
0.64077 ('BsmtFin SF 1', 'Bsmt Full Bath')
0.63277 ('SalePrice', 'Total Bsmt SF')
0.62758 ('Gr Liv Area', 'Full Bath')
0.62472 ('1st Flr SF', 'SalePrice')
0.61702 ('Half Bath', '2nd Flr SF')
0.60530 ('Overall Qual', 'Garage Cars')
0.60438 ('Year Remod/Add', 'Year Built')
0.59132 ('Year Built', 'Overall Qual')


In the most correlated pairs there's a lot of garage, year and area related features.
We will transform some of them to booleans and create a new feature 'Age

In [234]:
# make booleans for garage, basement, and 2nd floor
df["HasGarage"] = df["Garage Type"].notna()
df['HasBasement'] = df["Total Bsmt SF"] > 0
df['Has2ndFloor'] = df["2nd Flr SF"] > 0

# create the new feature 'age'
df['Age'] = df.apply(
    lambda x: x['Yr Sold']-x['Year Built'] if (
        x['Year Built'] < x['Year Remod/Add']
    ) else (
        x['Yr Sold']-x['Year Remod/Add']
    ), axis=1
)

In [236]:
px.scatter(x=df['Age'], y=df['SalePrice'], trendline='lowess', trendline_options={"frac": 0.1})

Looking at the trend we can see variations at some points of the trendline

We'll make bins corresponding to :
+ 0 to 5 years
+ 5 to 20 years
+ 20 to 50 years
+ 50+ years

In [237]:
bins = [-1, 5, 20, 50, 500]
df['AgeBins'] = pd.cut(df['Age'], include_lowest=True, bins=bins, labels=False)
del bins

In [238]:
# remove columns not used anymore
drop_columns = [
    'Garage Type',
    'Garage Yr Blt',
    'Garage Finish',
    'Garage Cars',
    'Garage Area',
    'Garage Qual',
    'Yr Sold',
    'Year Built',
    'Year Remod/Add',
    'Total Bsmt SF',
    'BsmtFin SF 1',
    'BsmtFin SF 2',
    'TotRms AbvGrd',
    'Age'
]

df = df.drop(drop_columns, axis=1)

del drop_columns

### Area above ground
'Gr Liv Area' is basically the sum of '1st Flr SF' and '2st Flr SF' so we only keep the first one.

In [239]:
df[['Gr Liv Area', '1st Flr SF', '2nd Flr SF']].head()
df = df.drop(['1st Flr SF', '2nd Flr SF'], axis=1)

### Bathrooms

In [240]:
df[['Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath', 'Half Bath']].head()


,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath
0,1.0,0.0,1,0
1,0.0,0.0,1,0
2,0.0,0.0,1,1
3,1.0,0.0,2,1
4,0.0,0.0,2,1


These features are grouped in a new feature 'bathrooms', corresponding to the total number of bathrooms

In [241]:
df["Bathrooms"] = df.apply(lambda x: x['Full Bath'] + x['Half Bath'] * .5, axis=1)
df = df.drop(['Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath', 'Half Bath'], axis=1)


### Fireplaces

A fireplace may have an impact on the sale price.

For convenience, we'll create a column "has_fireplace" and will remove "fireplaces" and "fireplace qu"


In [242]:
df["has_fireplace"] = df["Fireplaces"].apply(
    lambda x: 1 if x > 0 else 0)
df = df.drop(["Fireplaces", "Fireplace Qu"], axis=1)

### Quality features

In [243]:
df[["Overall Qual", "Overall Cond", "Exter Qual", "Exter Cond", "Functional"]]

,Overall Qual,Overall Cond,Exter Qual,Exter Cond,Functional
0,6,5,TA,TA,Typ
1,5,6,TA,TA,Typ
2,6,6,TA,TA,Typ
3,7,5,Gd,TA,Typ
4,5,5,TA,TA,Typ
...,...,...,...,...,...
2925,6,6,TA,TA,Typ
2926,5,5,TA,TA,Typ
2927,5,5,TA,TA,Typ
2928,5,5,TA,TA,Typ


No need to keep all of these since Overall quality combine them

In [244]:
df = df.drop(["Overall Cond", "Exter Qual", "Exter Cond", "Functional"], axis=1)

In [245]:
get_corr_pairs_thresh(df, 20, .5)

0.80032 ('Overall Qual', 'SalePrice')
0.71233 ('Gr Liv Area', 'SalePrice')
0.71146 ('Gr Liv Area', 'Bathrooms')
0.66322 ('Overall Qual', 'AgeBins')
0.59675 ('AgeBins', 'SalePrice')
0.57624 ('Bathrooms', 'SalePrice')
0.56930 ('Gr Liv Area', 'Overall Qual')
0.54834 ('Bathrooms', 'Overall Qual')
0.53990 ('Bathrooms', 'AgeBins')
0.52191 ('SalePrice', 'Mas Vnr Area')
0.51531 ('Bedroom AbvGr', 'Gr Liv Area')


We will keep these columns at the end.
+ __AgeBins__ : Bins created from Age of the houses (0-5, 5-20, 20-50, 50+ years)
+ __BldgType__ : Type of dwelling.
+ __GrLivArea__ : Above grade (ground) living area square feet.
+ __LotFrontage__ : Linear feet of street connected to property.
+ __LotArea__ : Lot size in square feet.
+ __Has2ndFloor__ : Has a 2nd floor or not.
+ __HasBasement__ : Has a basement or not.
+ __HasGarage__ : Has a garage or not.
+ __has_fireplace__: Has a fireplace or not
+ __Bedroom AbvGr__: Number of bedrooms
+ __Kitchen AbvGr__: Number of kitchens
+ __Bathrooms__: Number of bathrooms
+ __OverallQual__: Overall quality
+ __SalePrice__ : Target to predict

In [246]:
kept_cols = [
    'AgeBins', 
    'Bldg Type',
    'Gr Liv Area', 
    'Lot Frontage',
    'Lot Area',  
    'Has2ndFloor',
    'HasBasement',
    'HasGarage',
    'has_fireplace',
    'Bedroom AbvGr',
    'Kitchen AbvGr',
    'Bathrooms',
    'Overall Qual',
    'SalePrice'
]

## Handling missing values

In [247]:
df = df.replace(np.inf, np.nan)

In [248]:
df[kept_cols].isna().sum()

AgeBins            0
Bldg Type          0
Gr Liv Area        0
Lot Frontage     456
Lot Area           0
Has2ndFloor        0
HasBasement        0
HasGarage          0
has_fireplace      0
Bedroom AbvGr      0
Kitchen AbvGr      0
Bathrooms          0
Overall Qual       0
SalePrice          0
dtype: int64

We'll set missing values of 'LotFrontage' to the mean 'LotFrontage' of neighborhood

In [249]:
neighbors_mean_frontage = df["Lot Frontage"].groupby(df["Neighborhood"]).mean()

In [250]:
df["Lot Frontage"][df["Lot Frontage"].isna()] = df[df["Lot Frontage"].isna()].apply(lambda x: neighbors_mean_frontage.loc[x["Neighborhood"]], axis=1)

/tmp/ipykernel_48486/1916291674.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [251]:
df = df[kept_cols]

In [252]:
df.isna().sum()


AgeBins          0
Bldg Type        0
Gr Liv Area      0
Lot Frontage     3
Lot Area         0
Has2ndFloor      0
HasBasement      0
HasGarage        0
has_fireplace    0
Bedroom AbvGr    0
Kitchen AbvGr    0
Bathrooms        0
Overall Qual     0
SalePrice        0
dtype: int64

In [253]:
df = df.dropna()

In [254]:
df.columns

Index(['AgeBins', 'Bldg Type', 'Gr Liv Area', 'Lot Frontage', 'Lot Area',
       'Has2ndFloor', 'HasBasement', 'HasGarage', 'has_fireplace',
       'Bedroom AbvGr', 'Kitchen AbvGr', 'Bathrooms', 'Overall Qual',
       'SalePrice'],
      dtype='object')

## Change types of columns

In [255]:
cat_cols = [
    "AgeBins", "Bldg Type", "Has2ndFloor", 
    "HasBasement", "has_fireplace", "HasGarage", 
    "Bedroom AbvGr", "Kitchen AbvGr", "Bathrooms", "Overall Qual"
]
num_cols = ["Gr Liv Area", "Lot Frontage", "Lot Area"]
df[cat_cols] = df[cat_cols].astype("category")
df[num_cols] = df[num_cols].astype(int)
df["SalePrice"] = df ["SalePrice"].astype(int)
df.index = df.index.astype(int)
df = reduce_mem_usage(df)

Memory usage of dataframe is 0.130 MB
Memory usage after optimization is: 0.08 MB
Decreased by 39.3%


In [256]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2673 entries, 0 to 2929
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   AgeBins        2673 non-null   category
 1   Bldg Type      2673 non-null   category
 2   Gr Liv Area    2673 non-null   int16   
 3   Lot Frontage   2673 non-null   int16   
 4   Lot Area       2673 non-null   int32   
 5   Has2ndFloor    2673 non-null   category
 6   HasBasement    2673 non-null   category
 7   HasGarage      2673 non-null   category
 8   has_fireplace  2673 non-null   category
 9   Bedroom AbvGr  2673 non-null   category
 10  Kitchen AbvGr  2673 non-null   category
 11  Bathrooms      2673 non-null   category
 12  Overall Qual   2673 non-null   category
 13  SalePrice      2673 non-null   int32   
dtypes: category(10), int16(2), int32(2)
memory usage: 80.5 KB


In [257]:
df.head()

,AgeBins,Bldg Type,Gr Liv Area,Lot Frontage,Lot Area,Has2ndFloor,HasBasement,HasGarage,has_fireplace,Bedroom AbvGr,Kitchen AbvGr,Bathrooms,Overall Qual,SalePrice
0,2,1Fam,1656,141,31770,False,True,True,1,3,1,1.0,6,215000
1,2,1Fam,896,80,11622,False,True,True,0,2,1,1.0,5,105000
2,3,1Fam,1329,81,14267,False,True,True,0,3,1,1.5,6,172000
3,2,1Fam,2110,93,11160,False,True,True,1,3,1,2.5,7,244000
4,1,1Fam,1629,74,13830,True,True,True,1,3,1,2.5,5,189900


In [258]:
df.describe().applymap(lambda x: round(x))

,Gr Liv Area,Lot Frontage,Lot Area,SalePrice
count,2673,2673,2673,2673
mean,1508,70,10143,184680
std,502,22,8033,79049
min,334,21,1300,35000
25%,1141,60,7440,132000
50%,1456,70,9468,165000
75%,1755,80,11584,217000
max,5642,313,215245,755000


## Remove outliers

In [259]:
fig = px.histogram(df, x="SalePrice")
fig.show()

We notice a positive skewness, a management of extreme values ​​is necessary

In [260]:
continous_cols = df.select_dtypes(exclude=['category', 'O'])
continous_cols.describe().applymap(lambda x: round(x))

,Gr Liv Area,Lot Frontage,Lot Area,SalePrice
count,2673,2673,2673,2673
mean,1508,70,10143,184680
std,502,22,8033,79049
min,334,21,1300,35000
25%,1141,60,7440,132000
50%,1456,70,9468,165000
75%,1755,80,11584,217000
max,5642,313,215245,755000


In [261]:
for col in continous_cols:
    fig = px.histogram(df[col], title=f"Visualizing distribution for {col} column")
    fig.show()


We only keep values under the mean + 2 * standard deviation

In [270]:
df_out = df[(df["SalePrice"] < np.mean(df["SalePrice"]) + 2 * np.std(df["SalePrice"]))]
df_out = df_out[(df_out["Lot Area"] < np.mean(df["Lot Area"]) + 2 * np.std(df["Lot Area"]))]
df_out = df_out[(df_out["Gr Liv Area"] < np.mean(df["Gr Liv Area"]) + 2 * np.std(df["Gr Liv Area"]))]
df_out = df_out[(df_out["Lot Frontage"] < np.mean(df["Lot Frontage"]) + 2 * np.std(df["Lot Frontage"]))]
df_out.describe()

,Gr Liv Area,Lot Frontage,Lot Area,SalePrice
count,2402.000000,2402.000000,2402.000000,2402.000000
mean,1421.295171,66.418401,9179.743963,169788.636553
std,395.019641,17.100269,3603.991573,56693.243373
min,334.000000,21.000000,1300.000000,35000.000000
25%,1104.250000,59.000000,7200.000000,129500.000000
50%,1408.500000,70.000000,9100.000000,158000.000000
75%,1674.000000,77.000000,10941.250000,202500.000000
max,2504.000000,112.000000,26142.000000,341000.000000


In [271]:
for col in continous_cols:
    fig = px.histogram(df_out[col], title=f"Visualizing distribution for {col} column - removed outliers")
    fig.show()

In [272]:
df.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2673 entries, 0 to 2929
Columns: 14 entries, AgeBins to SalePrice
dtypes: category(10), int16(2), int32(2)
memory usage: 80.5 KB


In [273]:
df_out.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2402 entries, 1 to 2929
Columns: 14 entries, AgeBins to SalePrice
dtypes: category(10), int16(2), int32(2)
memory usage: 72.6 KB


In [276]:
df_out.reset_index(inplace=True)

In [277]:
df_out.to_csv('./data/02/cleaned_ames_full.csv')